In [120]:
from pypfopt import risk_models
from pypfopt import plotting
from pypfopt import expected_returns
from pypfopt import EfficientFrontier
from pypfopt import CLA, plotting
from pypfopt.black_litterman import BlackLittermanModel

import pypfopt

import yfinance as yf

from datetime import datetime
import matplotlib.pylab as plt
import pandas as pd

In [121]:
import plotly.express as px
import pandas
pandas.options.plotting.backend = "plotly"

In [225]:
def plot_allocation_performance(prices, weights):
    import numpy as np
    pct_change = prices.pct_change()[1:]
    performance = pd.Series(np.zeros(prices.shape[0] - 1), index=pct_change.index)
    holding_value = pd.DataFrame(data=np.zeros(pct_change.shape), index=pct_change.index, columns=pct_change.columns)
    for (ticker, weight) in weights.items():
        holding_value[ticker] = weight * (pct_change[ticker]+1).cumprod()
        performance = performance + holding_value[ticker]
    plt.plot(performance)

In [241]:
start_date = "2019-5-30"
tickers = ["PBW","DDOG","TWLO","FSLY","PTON","FIVN","CRWD","NVDA","ZM",
           "SE","TTD","ROKU","NET","ZS","OKTA","DOCU","SQ","NIO","FRPT","TRUP","CHWY","MDB","TSLA"]
ohlc = yf.download(tickers, period="max")
prices = ohlc["Adj Close"]
prices.tail()
prices = prices[start_date:]


[*********************100%***********************]  23 of 23 completed


In [251]:
def cov_shrinkage_opt_fn(prices, last_weights=None):
    mu = expected_returns.capm_return(prices)
    S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
    ef = EfficientFrontier(mu, S)
#     if last_weights is not None:
#         ef.add_objective(objective_functions.transaction_cost, w_prev=pd.Series(last_weights).values, k=0.003)
    ef.add_objective(objective_functions.L2_reg, gamma=0.1)  # gamme is the tuning parameter
    ef.min_volatility()
    return ef.clean_weights()
def semicovariance_opt_fn(prices, last_weights=None):
    mu = expected_returns.capm_return(prices)
    S = risk_models.semicovariance(prices)
    ef = EfficientFrontier(mu, S)
#     if last_weights is not None:
#         ef.add_objective(objective_functions.transaction_cost, w_prev=pd.Series(last_weights).values, k=0.003)
    ef.add_objective(objective_functions.L2_reg, gamma=0.1)  # gamme is the tuning parameter
    ef.min_volatility()
    return ef.clean_weights()
def exp_covariance_opt_fn(prices, last_weights=None):
    mu = expected_returns.capm_return(prices)
    S = risk_models.exp_cov(prices)
    ef = EfficientFrontier(mu, S)
#     if last_weights is not None:
#         ef.add_objective(objective_functions.transaction_cost, w_prev=pd.Series(last_weights).values, k=0.003)
    ef.add_objective(objective_functions.L2_reg, gamma=0.1)  # gamme is the tuning parameter
    ef.min_volatility()
    return ef.clean_weights()
def continuous_optimization(opt_fn, cov_lookback=120):
    import numpy as np
    start_idx = cov_lookback + 1
    assert(start_idx > cov_lookback)
    from pypfopt import EfficientFrontier, objective_functions
    prices_since_start = prices.iloc[start_idx:]
    returns = prices.pct_change()
    weight_history = pd.DataFrame(data=np.zeros(prices_since_start.shape), index=prices_since_start.index, 
                                  columns=prices_since_start.columns)
    starting_nav = 100000
    last_nav = starting_nav
    nav_history = pd.Series(data=np.zeros(prices_since_start.shape[0]) + last_nav, index=prices_since_start.index)
    last_weights = None

    for current_date_idx in range(start_idx, prices.shape[0]):
        timestamp = prices.index[current_date_idx]
        prices_to_date = prices[current_date_idx - cov_lookback:current_date_idx]
        #if current_date_idx == prices.shape[0] - 1:
        #    print("last timestamp's prices_to_date = ", prices_to_date)
        last_weights = opt_fn(prices_to_date, last_weights)
        weight_history.loc[timestamp,:] = last_weights
        if current_date_idx > start_idx:
            # Update the net asset value.
            last_nav = last_nav * (1+(returns.loc[timestamp] * pd.Series(last_weights)).sum())
            nav_history.loc[timestamp] = last_nav
    return {
        "nav_history": nav_history,
        "weight_history": weight_history
    }

In [242]:
r = continuous_optimization(cov_shrinkage_opt_fn)
r["nav_history"].plot()

last timestamp's prices_to_date =                   CHWY        CRWD        DDOG        DOCU        FIVN  \
Date                                                                    
2020-04-27  46.180000   73.599998   43.230000  108.250000   97.760002   
2020-04-28  44.160000   70.129997   43.599998  103.940002   96.220001   
2020-04-29  45.000000   69.279999   44.650002  103.400002   94.110001   
2020-04-30  43.240002   67.660004   45.119999  104.750000   92.669998   
2020-05-01  42.900002   69.209999   44.980000  103.519997   91.830002   
...               ...         ...         ...         ...         ...   
2020-10-08  58.669998  142.970001  102.660004  220.389999  136.380005   
2020-10-09  61.580002  143.570007  111.790001  225.600006  140.080002   
2020-10-12  64.449997  145.960007  112.199997  231.860001  142.630005   
2020-10-13  66.860001  152.800003  116.870003  238.210007  145.800003   
2020-10-14  64.120003  147.789993  113.550003  237.100006  144.669998   

               

In [243]:
r["weight_history"].plot()

In [203]:
r = continuous_optimization(exp_covariance_opt_fn)
r["nav_history"].plot()

In [204]:
r["weight_history"].plot()

In [201]:
r["weight_history"]

,CHWY,CRWD,DDOG,DOCU,FIVN,FRPT,FSLY,MDB,NET,NIO,...,PTON,ROKU,SE,SQ,TRUP,TSLA,TTD,TWLO,ZM,ZS
Date,,,,,,,,,,,,,,,,,,,,,
2019-11-19,0.09592,0.0,0.00000,0.07261,0.00047,0.14460,0.0,0.00000,0.04445,0.0,...,0.03319,0.00000,0.02882,0.0609,0.05944,0.03622,0.0,0.01844,0.02968,0.05111
2019-11-20,0.09592,0.0,0.00000,0.07261,0.00047,0.14460,0.0,0.00000,0.04445,0.0,...,0.03320,0.00000,0.02882,0.0609,0.05944,0.03622,0.0,0.01844,0.02968,0.05111
2019-11-21,0.09449,0.0,0.00000,0.07261,0.00047,0.14460,0.0,0.00000,0.04445,0.0,...,0.03330,0.00000,0.02882,0.0609,0.05944,0.03755,0.0,0.01844,0.02968,0.05111
2019-11-22,0.09379,0.0,0.00000,0.07261,0.00047,0.14460,0.0,0.00000,0.04445,0.0,...,0.03578,0.00000,0.02882,0.0609,0.05944,0.03755,0.0,0.01844,0.02968,0.05111
2019-11-25,0.09379,0.0,0.00000,0.07261,0.00047,0.14460,0.0,0.00000,0.04445,0.0,...,0.03715,0.00000,0.02882,0.0609,0.05944,0.03898,0.0,0.01844,0.02968,0.05111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-09,0.06848,0.0,0.05247,0.00000,0.12469,0.18936,0.0,0.04260,0.03133,0.0,...,0.00494,0.02613,0.00000,0.0000,0.08246,0.00000,0.0,0.03490,0.00000,0.00000
2020-10-12,0.06848,0.0,0.04287,0.00000,0.12469,0.18936,0.0,0.04535,0.02771,0.0,...,0.00000,0.02832,0.00000,0.0000,0.08246,0.00000,0.0,0.03682,0.00000,0.00000
2020-10-13,0.06848,0.0,0.04298,0.00000,0.12469,0.19105,0.0,0.05654,0.00064,0.0,...,0.00000,0.03681,0.00000,0.0000,0.08246,0.00000,0.0,0.03809,0.00000,0.00000


In [250]:
from pypfopt import expected_returns
from pypfopt import objective_functions
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from datetime import datetime, timedelta

import yfinance as yf
date_N_days_ago = datetime.now() - timedelta(days=175)
print("Fetching prices from %s to now" % date_N_days_ago)
prices = yf.download(tickers, period="max")["Adj Close"][date_N_days_ago:]
#print("Finished fetching prices, ", prices)
mu = expected_returns.capm_return(prices)
S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
ef = EfficientFrontier(mu, S)
ef.add_objective(objective_functions.L2_reg, gamma=0.1)  # gamme is the tuning parameter
ef.min_volatility()
weights = ef.clean_weights()
print(weights)


Fetching prices from 2020-04-24 02:31:29.694709 to now
[*********************100%***********************]  23 of 23 completed
OrderedDict([('CHWY', 0.09076), ('CRWD', 0.0207), ('DDOG', 0.05567), ('DOCU', 0.0), ('FIVN', 0.11246), ('FRPT', 0.13986), ('FSLY', 0.0), ('MDB', 0.05547), ('NET', 0.01745), ('NIO', 0.00396), ('NVDA', 0.08618), ('OKTA', 0.09252), ('PBW', 0.17485), ('PTON', 0.0109), ('ROKU', 0.04266), ('SE', 0.0), ('SQ', 0.0), ('TRUP', 0.08183), ('TSLA', 0.0), ('TTD', 0.00228), ('TWLO', 0.01247), ('ZM', 0.0), ('ZS', 0.0)])


In [132]:
from pypfopt import DiscreteAllocation

da = DiscreteAllocation(weights, prices.iloc[-1], total_portfolio_value=310000)
alloc, leftover = da.lp_portfolio()
print(f"Leftover: ${leftover:.2f}")
alloc

ValueError: Cannot broadcast dimensions  (21,) (22,)